In [2]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import torch.nn.functional as F

import matplotlib.pyplot as plt


In [3]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


## Data

In [4]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


## Basic Autoencoder

In [5]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [25]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')
    
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}_construct.png'.format(epoch))
        pic_org = to_img(img.cpu().data)
        save_image(pic_org, './mlp_img/image_{}_orignal.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

epoch [1/100], loss:0.2106
epoch [2/100], loss:0.1742
epoch [3/100], loss:0.1720
epoch [4/100], loss:0.1576
epoch [5/100], loss:0.1536
epoch [6/100], loss:0.1563
epoch [7/100], loss:0.1480
epoch [8/100], loss:0.1420
epoch [9/100], loss:0.1608
epoch [10/100], loss:0.1344
epoch [11/100], loss:0.1483
epoch [12/100], loss:0.1438
epoch [13/100], loss:0.1410
epoch [14/100], loss:0.1339
epoch [15/100], loss:0.1572
epoch [16/100], loss:0.1423
epoch [17/100], loss:0.1425
epoch [18/100], loss:0.1332
epoch [19/100], loss:0.1516
epoch [20/100], loss:0.1409
epoch [21/100], loss:0.1265
epoch [22/100], loss:0.1414
epoch [23/100], loss:0.1351
epoch [24/100], loss:0.1354
epoch [25/100], loss:0.1276
epoch [26/100], loss:0.1292
epoch [27/100], loss:0.1319
epoch [28/100], loss:0.1362
epoch [29/100], loss:0.1341
epoch [30/100], loss:0.1369
epoch [31/100], loss:0.1319
epoch [32/100], loss:0.1172
epoch [33/100], loss:0.1332
epoch [34/100], loss:0.1263
epoch [35/100], loss:0.1276
epoch [36/100], loss:0.1239
e

## VAE

In [37]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [39]:
if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


In [40]:
model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.item()/ len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}_construct.png'.format(epoch))
        pic_org = to_img(img.cpu().data)
        save_image(pic_org, './vae_img/image_{}_org.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

Train Epoch: 0 [0/60000 (0%)]	Loss: 1507.826050
Train Epoch: 0 [12800/60000 (21%)]	Loss: 734.697083
Train Epoch: 0 [25600/60000 (43%)]	Loss: 730.942566
Train Epoch: 0 [38400/60000 (64%)]	Loss: 728.547607
Train Epoch: 0 [51200/60000 (85%)]	Loss: 728.050232
====> Epoch: 0 Average loss: 742.1108
Train Epoch: 1 [0/60000 (0%)]	Loss: 725.793945
Train Epoch: 1 [12800/60000 (21%)]	Loss: 723.668091
Train Epoch: 1 [25600/60000 (43%)]	Loss: 721.803345
Train Epoch: 1 [38400/60000 (64%)]	Loss: 721.472839
Train Epoch: 1 [51200/60000 (85%)]	Loss: 719.504089
====> Epoch: 1 Average loss: 722.4908
Train Epoch: 2 [0/60000 (0%)]	Loss: 719.696533
Train Epoch: 2 [12800/60000 (21%)]	Loss: 716.679382
Train Epoch: 2 [25600/60000 (43%)]	Loss: 718.826294
Train Epoch: 2 [38400/60000 (64%)]	Loss: 716.230896
Train Epoch: 2 [51200/60000 (85%)]	Loss: 713.757080
====> Epoch: 2 Average loss: 716.8445
Train Epoch: 3 [0/60000 (0%)]	Loss: 714.261597
Train Epoch: 3 [12800/60000 (21%)]	Loss: 715.715332
Train Epoch: 3 [25600

Train Epoch: 27 [51200/60000 (85%)]	Loss: 705.359070
====> Epoch: 27 Average loss: 703.7963
Train Epoch: 28 [0/60000 (0%)]	Loss: 700.328125
Train Epoch: 28 [12800/60000 (21%)]	Loss: 704.752136
Train Epoch: 28 [25600/60000 (43%)]	Loss: 700.284668
Train Epoch: 28 [38400/60000 (64%)]	Loss: 700.743591
Train Epoch: 28 [51200/60000 (85%)]	Loss: 702.398804
====> Epoch: 28 Average loss: 703.7125
Train Epoch: 29 [0/60000 (0%)]	Loss: 702.336426
Train Epoch: 29 [12800/60000 (21%)]	Loss: 701.874695
Train Epoch: 29 [25600/60000 (43%)]	Loss: 706.012756
Train Epoch: 29 [38400/60000 (64%)]	Loss: 704.496643
Train Epoch: 29 [51200/60000 (85%)]	Loss: 699.797119
====> Epoch: 29 Average loss: 703.6074
Train Epoch: 30 [0/60000 (0%)]	Loss: 706.162964
Train Epoch: 30 [12800/60000 (21%)]	Loss: 706.637451
Train Epoch: 30 [25600/60000 (43%)]	Loss: 703.533447
Train Epoch: 30 [38400/60000 (64%)]	Loss: 703.263733
Train Epoch: 30 [51200/60000 (85%)]	Loss: 702.753845
====> Epoch: 30 Average loss: 703.5585
Train Epoch

Train Epoch: 55 [12800/60000 (21%)]	Loss: 701.906006
Train Epoch: 55 [25600/60000 (43%)]	Loss: 702.178528
Train Epoch: 55 [38400/60000 (64%)]	Loss: 700.590515
Train Epoch: 55 [51200/60000 (85%)]	Loss: 699.268066
====> Epoch: 55 Average loss: 702.5814
Train Epoch: 56 [0/60000 (0%)]	Loss: 704.613525
Train Epoch: 56 [12800/60000 (21%)]	Loss: 701.392517
Train Epoch: 56 [25600/60000 (43%)]	Loss: 700.866943
Train Epoch: 56 [38400/60000 (64%)]	Loss: 700.391602
Train Epoch: 56 [51200/60000 (85%)]	Loss: 705.014099
====> Epoch: 56 Average loss: 702.5017
Train Epoch: 57 [0/60000 (0%)]	Loss: 702.148499
Train Epoch: 57 [12800/60000 (21%)]	Loss: 706.019958
Train Epoch: 57 [25600/60000 (43%)]	Loss: 707.056274
Train Epoch: 57 [38400/60000 (64%)]	Loss: 704.306152
Train Epoch: 57 [51200/60000 (85%)]	Loss: 702.733154
====> Epoch: 57 Average loss: 702.5021
Train Epoch: 58 [0/60000 (0%)]	Loss: 702.657532
Train Epoch: 58 [12800/60000 (21%)]	Loss: 704.104797
Train Epoch: 58 [25600/60000 (43%)]	Loss: 697.8770

Train Epoch: 82 [51200/60000 (85%)]	Loss: 704.469727
====> Epoch: 82 Average loss: 702.0131
Train Epoch: 83 [0/60000 (0%)]	Loss: 700.361084
Train Epoch: 83 [12800/60000 (21%)]	Loss: 704.363281
Train Epoch: 83 [25600/60000 (43%)]	Loss: 701.547607
Train Epoch: 83 [38400/60000 (64%)]	Loss: 701.022400
Train Epoch: 83 [51200/60000 (85%)]	Loss: 706.203796
====> Epoch: 83 Average loss: 701.9786
Train Epoch: 84 [0/60000 (0%)]	Loss: 700.325012
Train Epoch: 84 [12800/60000 (21%)]	Loss: 700.926880
Train Epoch: 84 [25600/60000 (43%)]	Loss: 702.802429
Train Epoch: 84 [38400/60000 (64%)]	Loss: 702.912720
Train Epoch: 84 [51200/60000 (85%)]	Loss: 706.618591
====> Epoch: 84 Average loss: 701.9851
Train Epoch: 85 [0/60000 (0%)]	Loss: 704.218018
Train Epoch: 85 [12800/60000 (21%)]	Loss: 701.605103
Train Epoch: 85 [25600/60000 (43%)]	Loss: 700.262634
Train Epoch: 85 [38400/60000 (64%)]	Loss: 701.411926
Train Epoch: 85 [51200/60000 (85%)]	Loss: 706.411987
====> Epoch: 85 Average loss: 701.9650
Train Epoch

## Sparse Auto-encoder

In [8]:
model = autoencoder().cuda()
model_children = list(model.children())
if not os.path.exists('./sparse_img'):
    os.mkdir('./sparse_img')
    
# define the sparse loss function
def sparse_loss(autoencoder, images):
    loss = 0
    values = images
    for i in range(len(model_children)):
        values = F.relu((model_children[i](values)))
        loss += torch.mean(torch.abs(values))
    return loss

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [9]:
lamda = 0.001

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img) + lamda * sparse_loss(model, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './sparse_img/image_{}_construct.png'.format(epoch))
        pic_org = to_img(img.cpu().data)
        save_image(pic_org, './sparse_img/image_{}_orignal.png'.format(epoch))

torch.save(model.state_dict(), './sparse_autoencoder.pth')

epoch [1/100], loss:0.2067
epoch [2/100], loss:0.1818
epoch [3/100], loss:0.1690
epoch [4/100], loss:0.1428
epoch [5/100], loss:0.1605
epoch [6/100], loss:0.1581
epoch [7/100], loss:0.1494
epoch [8/100], loss:0.1470
epoch [9/100], loss:0.1334
epoch [10/100], loss:0.1534
epoch [11/100], loss:0.1403
epoch [12/100], loss:0.1242
epoch [13/100], loss:0.1436
epoch [14/100], loss:0.1261
epoch [15/100], loss:0.1439
epoch [16/100], loss:0.1440
epoch [17/100], loss:0.1425
epoch [18/100], loss:0.1376
epoch [19/100], loss:0.1127
epoch [20/100], loss:0.1230
epoch [21/100], loss:0.1251
epoch [22/100], loss:0.1429
epoch [23/100], loss:0.1370
epoch [24/100], loss:0.1265
epoch [25/100], loss:0.1302
epoch [26/100], loss:0.1316
epoch [27/100], loss:0.1371
epoch [28/100], loss:0.1298
epoch [29/100], loss:0.1279
epoch [30/100], loss:0.1345
epoch [31/100], loss:0.1321
epoch [32/100], loss:0.1255
epoch [33/100], loss:0.1430
epoch [34/100], loss:0.1424
epoch [35/100], loss:0.1187
epoch [36/100], loss:0.1297
e

In [11]:
! pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     |███████████████████             | 296.1 MB 153.4 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████▌    | 426.9 MB 112.2 MB/s eta 0:00:01

     |███████████████████████████▉    | 433.3 MB 178.9 MB/s eta 0:00:01

     |████████████████████████████▎   | 439.5 MB 178.9 MB/s eta 0:00:01

...

     |██████████████████████████████▊ | 477.4 MB 178.9 MB/s eta 0:00:01

     |███████████████████████████████ | 483.7 MB 178.9 MB/s eta 0:00:01

     |███████████████████████████████▌| 490.1 MB 178.9 MB/s eta 0:00:01

     |████████████████████████████████| 496.4 MB 178.9 MB/s eta 0:00:01

     |████████████████████████████████| 497.6 MB 7.6 kB/s s eta 0:00:0101
     |████████████████████████████████| 1.0 MB 121.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 123.7 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 118.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.1 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 4.1 MB/s s eta 0:00:01
     |████████████████████████████████| 462 kB 117.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 110.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 122.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 118.5 MB/s eta 0:00:01                       | 911 kB 118.5 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 112.5 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.0 MB/s s eta 0:00:01
     |████████████████████████████████| 4.9 MB 50.8 MB/s eta 0:00:01
     |██████████████████████████████